In [1]:
# imports & matplotlib options
%matplotlib notebook

#from IPython.core.display import display, HTML
import pyocr.builders
import pyocr
import pandas as pd
import numpy as np
import ipywidgets as widgets
import cv2
import collections
import random
from shutil import copy
from pathlib import Path
from dateutil.parser import parse
from datetime import datetime
from collections import OrderedDict
from autocorrect import spell
from fuzzywuzzy import fuzz, process
from ipywidgets import interact, IntProgress, Label, VBox
from IPython.display import display, HTML
from matplotlib import pyplot as plt
import img_qc.img_qc as img_qc
from PIL import Image


tools = pyocr.get_available_tools()
tool = tools[0]

plt.rc('figure', figsize=(25.0, 12.0))
display(HTML("<style>.container {width:85% !important;}</style>"))


print(f'\n{25 * "*_*"}\nLast run on {datetime.now().strftime("%Y-%m-%d %I:%M:%S %p")}')


*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Last run on 2019-09-03 11:22:05 AM


In [2]:
# pre-built lists to iterate over
months = [
    'january',
    'february',
    'march',
    'april',
    'may',
    'june',
    'july',
    'august',
    'september',
    'october',
    'november',
    'december'
]

roi_box_list = ['text', 'left', 'top', 'width', 'height']

print(f'\n{25 * "*_*"}\nLast run on {datetime.now().strftime("%Y-%m-%d %I:%M:%S %p")}')


*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Last run on 2019-09-03 11:22:05 AM


In [3]:
# test dictionary use
common_mistakes_dict = {'j': 't', 'd': 'j', 't': 'l', '£': 'a'}
for key in common_mistakes_dict:
    print(key)
    print('\t' + common_mistakes_dict[key])

j
	t
d
	j
t
	l
£
	a


In [4]:
# create a dictionary of possible month corrections
months_dict = {}
for month in months:
    correction_list = []
    for i in range(3):
        correction = month[i:]
        if len(correction) > 2:
            correction_list.append(month[i:])
    for i in range(1, 3):
        i = len(month) - i
        correction = month[:i]
        if len(correction) > 2:
            correction_list.append(month[:i])
    # print(correction_list)
    # add replacement options with FIND: REPLACE
    # NOTE: depending on order pulled will get different dictionary
    # because downstream letter replacements will have more options to change
    # example: july -> tuly -> luly wouldn't happen if ('j':'l') came before ('j':'t')
    errors_dict = {'j': 't', 'd': 'j', 't': 'l', 'a': '£', 'f': '£', 'a': '&',}
    for error in errors_dict:
        for correction in correction_list:
            if error in correction:
                # print(error, errors_dict[error])
                replaced = correction.replace(error, errors_dict[error])
                # print(replaced)
                correction_list.append(replaced)
        
    months_dict[month] = correction_list
# the first 2 letters can be missing
for month in months_dict:
    print(f'{month}\n{months_dict[month]}\n')

january
['january', 'anuary', 'nuary', 'januar', 'janua', 'tanuary', 'tanuar', 'tanua', 'lanuary', 'lanuar', 'lanua', 'j&nu&ry', '&nu&ry', 'nu&ry', 'j&nu&r', 'j&nu&', 't&nu&ry', 't&nu&r', 't&nu&', 'l&nu&ry', 'l&nu&r', 'l&nu&']

february
['february', 'ebruary', 'bruary', 'februar', 'februa', 'febru&ry', 'ebru&ry', 'bru&ry', 'febru&r', 'febru&', '£ebruary', '£ebruar', '£ebrua', '£ebru&ry', '£ebru&r', '£ebru&']

march
['march', 'arch', 'rch', 'marc', 'mar', 'm&rch', '&rch', 'm&rc', 'm&r']

april
['april', 'pril', 'ril', 'apri', 'apr', '&pril', '&pri', '&pr']

may
['may', 'm&y']

june
['june', 'une', 'jun', 'tune', 'tun', 'lune', 'lun']

july
['july', 'uly', 'jul', 'tuly', 'tul', 'luly', 'lul']

august
['august', 'ugust', 'gust', 'augus', 'augu', 'augusl', 'ugusl', 'gusl', '&ugust', '&ugus', '&ugu', '&ugusl']

september
['september', 'eptember', 'ptember', 'septembe', 'septemb', 'seplember', 'eplember', 'plember', 'seplembe', 'seplemb']

october
['october', 'ctober', 'tober', 'octobe', 'oc

In [7]:
# create page 1 paths list
identifier = 'agrutesc'
data_dir_path = Path('data/')
page_1_paths_list = sorted(data_dir_path.joinpath('images', identifier).glob('*.tif'))

# delete macOS '.' index files
regenerate_paths_list = False
for path in page_1_paths_list:
    if path.name.startswith('.'):
        path.unlink()
        regenerate_paths_list = True

if regenerate_paths_list:
    page_1_paths_list = sorted(data_dir_path.joinpath('images', identifier).glob('*.tif'))

print(f'{len(page_1_paths_list)} images in page 1 paths list')


print(f'\n{25 * "*_*"}\nLast run on {datetime.now().strftime("%Y-%m-%d %I:%M:%S %p")}')

217 images in page 1 paths list

*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Last run on 2019-09-03 11:23:49 AM


In [8]:
# utility functions
def crop_image_for_processing(image, percentage=0.4, top_and_sides_padding=10):

    # get technical metadata
    height, width = image.shape[:2]

    # set (x, y) pairs
    x1, y1, x2, y2 = 0, 0, width, int(height * percentage)

    # add/subtract padding from the top/left/right
    x1 += top_and_sides_padding
    y1 += top_and_sides_padding
    x2 -= top_and_sides_padding
    # add image area to the bottom that was cropped from the top
    y2 += top_and_sides_padding

    # crop image
    image = image[y1:y2, x1:x2]

    return image


def get_np_crop_points(crop_box):
    x_points = []
    y_points = []
    # print(f'crop box: {crop_box}')

    # append all x/y points to their respective lists
    for i in range(len(crop_box)):
        if crop_box[i][0][0]:
            x = (crop_box[i][0][0])
            if x < 0:
                x = -x
            x_points.append(x)
        if crop_box[i][0][1]:
            y = crop_box[i][0][1]
            if y < 0:
                y = -y
            y_points.append(y)
    # print('x/y points')
    # print(x_points, y_points)

    # find extremes in crop box
    x1 = min(x_points)
    x2 = max(x_points)
    y1 = min(y_points)
    y2 = max(y_points)
    # print(f'x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}')

    return x1, y1, x2, y2


def if_rgb_convert_to_gray(np_image):
    if len(np_image.shape) > 2:
        np_image = cv2.cvtColor(np_image, cv2.COLOR_RGB2GRAY)

    return np_image


def if_bgr_convert_to_gray(np_image):
    if len(np_image.shape) > 2:
        np_image = cv2.cvtColor(np_image, cv2.COLOR_BGR2GRAY)

    return np_image


def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        ratio = height / float(h)
        dim = (int(w * ratio), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        ratio = width / float(w)
        dim = (width, int(h * ratio))

    # resize the image
    resized = cv2.resize(image, dim, interpolation=inter)

    # return the resized image
    return resized, ratio


def resize_ratio(image, ratio, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    dim = (int(w * ratio), int(h * ratio))

    # resize the image
    resized = cv2.resize(image, dim, interpolation=inter)

    # return the resized image
    return resized


def quick_imshow(bgr_image):
    if len(bgr_image.shape) > 2:
        bgr_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
        cmap = None
    else:
        cmap = 'gray'
    plt.imshow(bgr_image, cmap=cmap), plt.show()
    
    return


print(f'\n{25 * "*_*"}\nLast run on {datetime.now().strftime("%Y-%m-%d %I:%M:%S %p")}')


*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Last run on 2019-09-03 11:23:49 AM


In [9]:
class TnFarmNews:

    def __init__(self, image_path, config=False):

        # instantiate image_path and load image
        self.image_path = Path(image_path)
        self.image = cv2.imread(str(self.image_path))

        # set config dictionary
        if config:
            self.config = config
        else:
            self.config = {'crop_percentage': 0.4,
                           'distance_ratio': 0.6,
                           'minimum_matches': 5,
                           'top_and_bottom_padding': 10,
                           'resize_width': 1000}

        # set variables
        self.MOST_MATCHES = self.config['minimum_matches']
        self.BEST_SSIM_WITH_TITLE = -1  # on a scale of -1->1, 1 is a perfect match
        self.BEST_TITLE_CROP = False
        self.BEST_TITLE = False
        self.RESIZE_RATIO = False

    def preprocess_image(self, crop_percentage=None, top_and_bottom_padding=None):

        if not crop_percentage:
            crop_percentage = self.config['crop_percentage']
        if not top_and_bottom_padding:
            top_and_bottom_padding = self.config['top_and_bottom_padding']

        # get technical metadata
        self.height, self.width = self.image.shape[:2]
        if len(self.image.shape) > 2:
            self.colorspace = 'gray'
        else:
            self.colorspace = 'rgb'

        # crop image and get shape
        self.image_cropped = crop_image_for_processing(
            self.image, crop_percentage, top_and_bottom_padding)
        self.height_cropped, self.width_cropped = self.image_cropped.shape[:2]

        return self.image_cropped

    def guess_title(self, crop_dictionary, image=None, debug=None):

        print(f'Guessing title of {self.image_path.name} . . .')

        if not image:
            image = self.preprocess_image()

        self.keypoints, self.descriptors = get_keypoints_and_descriptors(image)

        best_per_title_dict = OrderedDict()

        for title, data in crop_dictionary.items():

            if title not in best_per_title_dict:  # instantiate an OrderedDict()
                # number_of_matches, crop_box
                best_per_title_dict[title] = OrderedDict()
                best_per_title_dict[title] = [0, False, False, False]
                if debug:
                    print(title)

            for sub_title, sub_data in crop_dictionary[title].items():

                crop_image, crop_keypoints, crop_descriptors = sub_data
                # quick_imshow(crop_image)

                number_of_matches, matches = get_descriptor_matches(
                    crop_descriptors, self.descriptors, self.config['distance_ratio'])
                if debug:
                    print(sub_title)
                    print(
                        f'# of matches/minMatches: {number_of_matches}/{(self.MOST_MATCHES * 0.5)}')

                # then it might be the best match
                if number_of_matches >= (self.MOST_MATCHES * 0.5):

                    found_image, crop_box = get_matching_crop_and_box(
                        matches, crop_keypoints, self.keypoints, crop_image, image)

                    try:
                        if not crop_box:  # then we didn't get a valid crop_box
                            if debug:
                                print(f'\t\tFalse positive: invalid crop_box')
                            continue
                    except ValueError:  # valid numpy array throws a ValueError when testing existence

                        if debug:
                            print(f'crop box: {crop_box}')
                        x1, y1, x2, y2 = get_np_crop_points(crop_box)

                        # x1 = int(x1 / self.RESIZE_RATIO)
                        # y1 = int(y1 / self.RESIZE_RATIO)
                        # x2 = int(x2 / self.RESIZE_RATIO)
                        # y2 = int(y2 / self.RESIZE_RATIO)

                        # if width/height of title isn't over min_title_width/height pixels
                        # it's not the title
                        min_title_width = 800
                        min_title_height = 40
                        title_width = x2 - x1
                        title_height = y2 - y1

                        if (x2 - x1 < min_title_width):
                            if debug:
                                print(f'\t\t\tFalse positive: width < minimum')
                                print(
                                    f'\t\t\t\t{x2} - {x1} = {title_width} < {min_title_width}')
                            continue
                        elif (y2 - y1 < min_title_height):
                            if debug:
                                print(f'\t\t\tFalse positive: height < minimum')
                                print(
                                    f'\t\t\t\t{y2} - {y2} = {title_height} < {min_title_height}')
                            continue

                        if debug:
                            print(x1, y1, x2, y2)
                            crop = self.image_cropped[int(
                                y1):int(y2), int(x1):int(x2)]
                            print(f'cropped image')
                            quick_imshow(crop)

                        # convert to grayscale for ssim
                        found_gray = if_bgr_convert_to_gray(found_image)
                        crop_gray = if_bgr_convert_to_gray(crop_image)

                        # get the structural similiarity index of the match with the image for a 2nd heuristic
                        try:
                            if debug:
                                print('found_gray')
                                quick_imshow(found_gray)
                                print(f'crop_gray')
                                quick_imshow(crop_gray)
                            ssim_value = ssim(found_gray, crop_gray)
                            if ssim_value > self.BEST_SSIM_WITH_TITLE:
                                print(f'new best ssim: {title, ssim_value}')
                                self.BEST_SSIM_WITH_TITLE = ssim_value
                                points = [x1, y1, x2, y2]
                                self.BEST_TITLE_CROP = points
                                self.BEST_TITLE = title
                                self.MOST_MATCHES = number_of_matches

                        except ValueError:  # image shape doesn't match, so most likely incorrect title
                            print('wrong dimensions')
                            continue
        return

    def binarize(self, image=None, debug=None):

        if type(image) is not np.ndarray:
            image = self.preprocess_image()

        # convert to gray and blur
        image = if_bgr_convert_to_gray(image)
        image = cv2.GaussianBlur(image, (3, 3), 1)

        # binarize
        self.binarized = cv2.adaptiveThreshold(
            image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 175, 21)
        
        return self.binarized

    def ocr(self, image=None, debug=None):
        
        if type(image) is not np.ndarray:
            image = self.binarize()
        
        # show image we're OCRing
        # quick_imshow(image)
        
        # create line and word boxes
        line_and_word_boxes = tool.image_to_string(
            Image.fromarray(image),
            lang='eng',
            builder=pyocr.builders.LineBoxBuilder(),
        )
        # create text
        text = tool.image_to_string(
            Image.fromarray(image),
            lang='eng',
            builder=pyocr.builders.TextBuilder(),
        )

        return line_and_word_boxes, text
    
    def box_crop(self, crop_box, image=None):
        
        print(f'crop_box: {crop_box}')
        # set crop_box
        x1, y1, x2, y2 = crop_box

        try:  # cropping the image
            self.image_box_crop = image[y1:y2, x1:x2]
        except TypeError:  # because it's *probably* a NoneType
            # load image from class
            self.image_box_crop = self.image[y1:y2, x1:x2]
        
        return self.image_box_crop
    
    def draw_box(self, image, crop_box):
    
        x1, y1, x2, y2 = crop_box

        # draw rectangle
        self.drawing = cv2.rectangle(image.copy(), (x1, y1), (x2, y2), (0, 0, 255), 2)

        return self.drawing
    
    def ocr_thrice(self, image=None, debug=None):
        
        if type(image) is not np.ndarray:
            image = self.binarize()
            
        height, width = image.shape[:2]
        
        # ocr everything
        self.ocr_all = self.ocr(image)
        self.ocr_all_lines = self.ocr_all[0]
        self.ocr_all_text = self.ocr_all[1]
        
        # crop and ocr right half
        image = image[0:height, int(width/2):width]
        self.ocr_right = self.ocr(image)
        self.ocr_right_lines = self.ocr_right[0]
        self.ocr_right_text = self.ocr_right[1]
        
        # crop and ocr left half
        image = image[0:height, 0:int(width/2)]
        self.ocr_left = self.ocr(image)
        self.ocr_left_lines = self.ocr_left[0]
        self.ocr_left_text = self.ocr_left[1]
        
        # return ocr_all
        return self.ocr_all
        

print(f'\n{25 * "*_*"}\nLast run on {datetime.now().strftime("%Y-%m-%d %I:%M:%S %p")}')


*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Last run on 2019-09-03 11:23:50 AM


In [10]:
# ocr functions/class
def search(values, searchFor):
    for k in values:
        for v in values[k]:
            if searchFor in v:
                return k
    return None


class OcrResults:
    def __init__(self, line_and_word_boxes):
        self.filter_chars = 'abcdefghijklmnopqrstuvwxyz1234567890£& '
        self.line_and_word_boxes = line_and_word_boxes

    def get_month(self, debug=False):
        
        self.months = OrderedDict()
        number_of_months = 0

        # iterate over lines in the OCR result
        for line in self.line_and_word_boxes:
            # filter out everything that's not a a-z,0-9, and a 'space'
            line_condensed = [
                x for x in line.content if x.lower() in self.filter_chars]

            # filtering produces a list of chars, so join it into a line of text
            text = ''.join(line_condensed)

            # split the joined text into words at each space
            for word in text.split(' '):

                # only process word results over 2 chars in length
                if len(word) > 2:

                    # try to match the word as lower-case with the months dictionary
                    month = search(months_dict, word.lower())

                    if month is None:  # try autocorrecting the word and searching again
                        fixed_word = spell(word)
                        if debug:
                            print(f'fixed_word: {fixed_word}')
                        month = search(months_dict, fixed_word.lower())

                    if month is None:  # continue on to the next word
                        continue
                        
                    # get the word 
                    for index, box in enumerate(line.word_boxes):
                        if box.content == word:
                            found_month = month
                            # print(box.content)
                            # print(box.position)
                            if debug:
                                print(f'index: {index}')
                                if index > 0:
                                    word_before = line.word_boxes[index-1].content
                                    print(f'word before: {word_before}')
                                word_found = line.word_boxes[index].content
                                print(f'word: {word_found}')
                                word_after = line.word_boxes[index+1].content
                                print(f'word after: {word_after}')
                            (x1, y1), (x2, y2) = box.position[:2]
                            crop_box = [x1, y1, x2, y2]
                            number_of_months += 1
                            yield found_month, word, crop_box
        # otherwise
        if number_of_months > 0:
            return
        else:
            yield None

print(f'\n{25 * "*_*"}\nLast run on {datetime.now().strftime("%Y-%m-%d %I:%M:%S %p")}')


*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Last run on 2019-09-03 11:23:51 AM


In [ ]:
string = 'string'
string = ' '.join(string)

In [ ]:
# string = 'string'
print(string)
for index, char in enumerate(string.split(' ')):
    print(f'index: {index}')
    print(f'\tchar: "{char}"')
    print(f'\tslice list by index +1:')
    print(f'\t\t\t\tstring[{index+1:}:]: {" ".join(string.split(" ")[index+1:])}')

In [11]:
# make sure we're saving the CSV locally as images are not
csv_path = data_dir_path.joinpath('agrutesc_date.csv')
if csv_path.is_file():
    processed_df = pd.read_csv(str(csv_path))
else:
    processed_df = pd.DataFrame()
processed_df

,date_crop_box,date_guess,date_guess_id,image_name
0,"(2616, 225, 3270, 334)","Ji anuary 27 , 33",1,0012_004268_000001_0001.tif
1,"(212, 229, 754, 337)",October 1539,1,0012_004268_000004_0001.tif
2,"(2439, 540, 3029, 652)",Apr-32,2,0012_004268_000004_0001.tif
3,"(2605, 561, 3261, 657)",Oct-32,1,0012_004268_000005_0001.tif
4,"(2449, 195, 3037, 289)",Apr-38,1,0012_004268_000006_0001.tif
5,"(2542, 277, 3132, 368)",Apr-38,1,0012_004268_000007_0001.tif
6,"(2399, 581, 3117, 685)",Sep-33,1,0012_004268_000008_0001.tif
7,"(2469, 796, 3185, 906)",Sep-33,1,0012_004268_000009_0001.tif
8,"(378, 270, 919, 376)",Oct-40,1,0012_004268_000010_0001.tif
9,"(1706, 816, 2230, 907)",Mar-34,2,0012_004268_000010_0001.tif


In [12]:
# batch process text

rows_list = []

images_to_process_before_saving_csv = 20

# paths_list = [Path('data/images/0012_004266_000331_0001.tif')]
# paths_list = random.sample(page_1_paths_list, len(page_1_paths_list))
paths_list = page_1_paths_list
number_of_paths = len(paths_list)

# progress bar
progress_label = Label('Images to process')
progress_bar = IntProgress(min=0, max=number_of_paths)
progress_widget = VBox([progress_label, progress_bar])
display(progress_widget)


for index, image_path in enumerate(paths_list, start=1):

    if len(processed_df) > 0:  # if the dataframe has any data
        # check the path field to for the current image path and
        if (processed_df['image_name'] == str(image_path.name)).any():  # if it exists
            # if modulo of processed images is 0 or it's the last image save data to the CSV
            if (index % images_to_process_before_saving_csv) == 0 or index == number_of_paths:

                print(f'Saving data from {index} images to {csv_path}')

                # get dataframe from processed rows
                crop_df = pd.DataFrame(rows_list)

                # add dataframes together
                processed_df = pd.concat([processed_df, crop_df])

                # drop duplicates
                processed_df.drop_duplicates(inplace=True)

                # sort on image_path and reset the index
                processed_df = processed_df.sort_values(
                    by=['image_name', 'date_guess_id']).reset_index(drop=True)
                processed_df.to_csv(csv_path, index=False)

                # reset rows_list
                rows_list = []
            continue  # on to the next image_path

    label = f'Processing {image_path.name} . . . {index+1}/{number_of_paths}'
    progress_label.value = label

    # process image
    issue = TnFarmNews(image_path)
    # print(issue.image_path.name)
    issue.ocr_thrice()
    image = issue.image_cropped
    # quick_imshow(image)
    height, width = issue.height_cropped, issue.width_cropped

    ocr_results_dict = {'all': issue.ocr_all_lines,
                        'left': issue.ocr_left_lines,
                        'right': issue.ocr_right_lines}

    crop_and_text_dict = OrderedDict()

    for area_ocred in ocr_results_dict:

        # print(f'OCRing image area "{area_ocred}" . . .')

        # instantiate OcrResults class to find month
        ocr_results = OcrResults(ocr_results_dict[area_ocred])

        # loop through results
        for i, result in enumerate(ocr_results.get_month()):
            # print(f'Result: {i + 1}\n{25 * "="}')
            if result is None:
                # print(issue.text)
                # print('\t\t\t\tNo acceptable results\n')
                continue
            else:
                month, word, crop_box = result
                # print(f'\t\t\t\t{month.capitalize(), word}')
                x1, y1, x2, y2 = crop_box

                if area_ocred == 'right':
                    cropped_width = int(width/2)
                    # add cropped back to x1 and x2
                    x1 += cropped_width
                    x2 += cropped_width
                # print(f'month crop_box: {crop_box}')
                # quick_imshow(image[y1:y2, x1:x2])

                # get larger crop_box and re-OCR
                x1 -= 25
                x1 = max(x1, 0)
                y1 -= 25
                y1 = max(y1, 0)
                x2 += 400
                x2 = min(x2, width)
                y2 += 25
                y2 = min(y2, height)

                # draw this box on larger image
                # drawing = cv2.rectangle(
                #     image.copy(), (x1, y1), (x2, y2), (0, 0, 255), 8)
                # print('expanded month crop box on image')
                # quick_imshow(drawing)

                # crop to sub_image and ocr
                sub_image = image[y1:y2, x1:x2]
                sub_image = issue.binarize(sub_image)
                line_and_word_boxes, text = issue.ocr(sub_image)

                # print(f'Text:\n"{text}"\n\n')
                # print(f'{len(line_and_word_boxes)} lines in sub_image')

                # package sub_image's crop_box for dictionary
                date_crop_box = (x1, y1, x2, y2)

                # it's possible there's more than 1 line of text
                # so make each one a possible date
                for line in line_and_word_boxes:
                    # print(line.content)

                    # replace new lines with spaces
                    line = line.content.replace('\n', ' ')
                    # print(line)
                    # strip periods, single/double-quotes, and spaces at the beginning and end of the line
                    line = line.strip('.').strip(' ').strip("'").strip('"')
                    # print(line)

                    # many of the lines are obviously not dates
                    # so only add lines/crop_boxes if
                    # the line contains any numbers
                    for char in line:
                        if char.isdigit():
                            crop_and_text_dict[line] = date_crop_box
                            break

    # process OCR results

    count_for_csv = 1

    for date in crop_and_text_dict:

        parsed_date = None
        month = None
        date_guess = None

        if len(date) < 5:
            continue
        try:
            # print(f'Processing: "{date}"')
            parsed_date = parse(date)

        except ValueError:  # couldn't parse the date
            # split on spaces and look for each "word" in the months dictionary
            words = date.split(' ')

            for i, word in enumerate(words):

                word = str(word)  # cast word as a string in case of weirdness

                # only process word results over 2 chars in length
                if len(word) > 2:

                    best_word = word

                    # try to match the word as lower-case with the months dictionary
                    # returns None on failed search
                    month = search(months_dict, word.lower())

                    if month is None:  # try autocorrecting the word and searching again
                        fixed_word = spell(word)
                        # print(f'fixed_word: {fixed_word}')
                        month = search(months_dict, fixed_word.lower())
                        best_word = fixed_word

                    # try adding the previous word to every word after the first and re-searching
                    if i > 0 and month is None:
                        word_plus_word_before = words[i-1] + word
                        month = search(
                            months_dict, word_plus_word_before.lower())
                        best_word = word_plus_word_before

                    # try adding the next word to every word but the last
                    if i+1 < (len(words)) and month is None:
                        word_plus_word_after = word + words[i+1]
                        month = search(
                            months_dict, word_plus_word_after.lower())
                        best_word = word_plus_word_after

                    if month is None:  # continue to the next word
                        continue
                    else:
                        # if debug:
                        #     print(
                        #         f'word guess for {month} from best word: {best_word}')

                        # set the year to be every word after the end of the month
                        # joining them with a space
                        year = ' '.join(words[i+1:])

                        # TODO: Could validate the year here

                        corrected_date = f'{month} {year}'
                        # if debug:
                        #     print(f'after: {corrected_date}')

                        try:  # parsing the corrected date
                            
                            parsed_date = parse(corrected_date)
                            # if debug:
                            #     print(f'parsed corrected date: {parsed_date}')
                                
                        except ValueError:  # it didn't work
                            
                            try:  # try parsing it again with a fuzzy corrected date
                                
                                parsed_date = parse(corrected_date, fuzzy=True)
                                # if debug:
                                #     print(
                                #         f'fuzzy parsed corrected date: {parsed_date}')
                                    
                            except ValueError:  # yup, failed again, sorry, chief
                                
                                # if debug:
                                #     parsed_date = "Date couldn't be parsed"
                                continue  # on to the next "date" we found

        if parsed_date is None:
            # if we found a month go ahead and post the text value
            # of the found date as it may be correct
            # print(f'month in parsed_data is None: {month}')
            if month:
                date_guess = str(date)
            else:
                date_guess = None

        # if date after 2000 just pass the text
        elif int(parsed_date.strftime("%Y")) > 1999:
            date_guess = str(date)
        #     if month:
        #         date_guess = str(date)
        #     else:
        #         date_guess = None

        else:
            # print(f'Type of parsed_date: {type(parsed_date)}')
            date_guess = parsed_date.strftime("%B %Y")
            month = parsed_date.strftime("%B")
            year = parsed_date.strftime("%Y")
            # if first digit of year is 0, change it to 1
            if int(year[0]) == 0:
                # print(f'year <1000: {year}')
                year = '1' + year[1:]
                # print(f'year fixed: {year}')
                # then rebuild date_guess
                date_guess = f'{month} {year}'
            elif int(year[1]) != 9:
                year = '19' + year[2:]
                # then rebuild date_guess
                date_guess = f'{month} {year}'

        crop_box = crop_and_text_dict[date]

        x1, y1, x2, y2 = crop_box

#         # draw this box on the ocr_all image
#         drawing = cv2.rectangle(image.copy(), (x1, y1), (x2, y2), (0, 0, 255), 8)
#         # draw a black rectangle above this to add text to
#         drawing = cv2.rectangle(drawing, (x1, y1-100), (x2, y1), (0, 0, 0), -1)  # negative thickness to fill rectangle

#         font = cv2.FONT_HERSHEY_SIMPLEX
#         print(f'date_guess before fail: {date_guess}')
#         if date_guess is not None:
#             date_writing = date_guess
#         else:
#             date_writing = 'Date Guess Fail'
#         cv2.putText(drawing, date_writing, (x1+25, (y1-25)), font, 2, (255,255,255), 3, cv2.LINE_AA)

#         quick_imshow(drawing)

        # get input row in dictionary format
        # key = column_name
        results_dictionary = {'date_guess_id': count_for_csv,
                              'image_name': image_path.name,
                              'date_guess': date_guess,
                              'date_crop_box': crop_box
                              }

        for key in results_dictionary:
            if results_dictionary[key]:
                continue
            else:
                results_dictionary.update({key: None})

        rows_list.append(results_dictionary)
        # increment counter for imageID
        count_for_csv += 1

    progress_bar.value = index

    # if modulo of processed images is 0 or it's the last image save data to the CSV
    if (index % images_to_process_before_saving_csv) == 0 or index == number_of_paths:

        print(f'Saving data from {index} images to {csv_path}')

        # get dataframe from processed rows
        crop_df = pd.DataFrame(rows_list)

        # add dataframes together
        processed_df = pd.concat([processed_df, crop_df])

        # drop duplicates
        processed_df.drop_duplicates(inplace=True)

        # sort on image_path and reset the index
        processed_df = processed_df.sort_values(
            by=['image_name', 'date_guess_id']).reset_index(drop=True)
        processed_df.to_csv(csv_path, index=False)

        # reset rows_list
        rows_list = []

Saving data from 20 images to data/agrutesc_date.csv
Saving data from 40 images to data/agrutesc_date.csv
Saving data from 60 images to data/agrutesc_date.csv
Saving data from 80 images to data/agrutesc_date.csv
Saving data from 100 images to data/agrutesc_date.csv
Saving data from 120 images to data/agrutesc_date.csv
Saving data from 140 images to data/agrutesc_date.csv
Saving data from 160 images to data/agrutesc_date.csv


KeyboardInterrupt: 

In [ ]:
# get dataframe from processed rows
crop_df = pd.DataFrame(rows_list)

# add dataframes together
processed_df = pd.concat([processed_df, crop_df], sort=False)

# drop duplicates
processed_df.drop_duplicates(subset=['image_id'], inplace=True)

# sort on image_path and reset the index
processed_df = processed_df.sort_values(by='image_id').reset_index(drop=True)
processed_df.to_csv(csv_path, index=False)

In [ ]:
# # process and load an image and process to be displayed
# issue = TnFarmNews(page_1_paths_list[22])
# image_name = .name
# path = page_1_paths_dictionary[image_name]
# image_key = path.name
# image = cv2.imread(str(path))
# image_resized = resize(image, )
# image_resized = image_resized.resize((int(width*0.25), int(height*0.25)))
# image_resized_cv2 = np.array(image_resized)
# image_resized_cv2 = np.stack((image_resized_cv2,)*3, axis=-1)

In [ ]:
# create pop-up image and choose where to OCR on it -- backup
from matplotlib.widgets import Button

# create matplotlib figure and axis
fig, ax = plt.subplots()

# subplot the figure with gridspec
gs = fig.add_gridspec(10,10)
image_ax = fig.add_subplot(gs[0:8, :])
previous_ax = fig.add_subplot(gs[9, 8])
next_ax = fig.add_subplot(gs[9, 9])

# issue.preprocess_image()
#     image_resized = Image.fromarray(issue.image).convert('L')
#     width, height = image_resized.size
#     image_resized = image_resized.resize((int(width*0.25), int(height*0.25)))
#     image_resized = np.array(image_resized)
#     image_resized = np.stack((image_resized,)*3, axis=-1)
#     image_ax.imshow(image_resized, cmap='gray')
#     fig.canvas.draw()
    
#     return image_resized

def resize_issue_for_qt_popup(issue):
    image_resized, resize_ratio = resize(issue.image, width=issue.config['resize_width'])
    return image_resized, resize_ratio
    
    
# initial data
page_1_paths_dictionary = {path.name: path for path in page_1_paths_list}
    
      

# Create widgets
image_name_widget = widgets.Dropdown(
    options=list(page_1_paths_dictionary), value=image_key)
bprev = Button(previous_ax, 'Previous', hovercolor='green')
bnext = Button(next_ax, 'Next', hovercolor='green')

image_name_widget.observe(choose_image, 'value')



interact(choose_image, image_name=[x.name for x in page_1_paths_list])

def choose_image(image_name):
    # Let's store the result in the global `image` that we can then use in our image editor below
    global image_resized, resize_ratio
    try:
        image_path = page_1_paths_dictionary[image_name.new]
    except AttributeError:  # 'str' object has no attribute 'new' if it's the first image loaded
        image_path = page_1_paths_dictionary[image_name]
    
    issue = TnFarmNews(image_path)
    # print(issue.image_path.name)
    
    image_resized, resize_ratio = resize_issue_for_qt_popup(issue)
    
    image_ax.imshow(image_resized, cmap='gray')
    
    fig.canvas.draw()
    
    return image_resized, resize_ratio


def create_crop_box_from_click_data(xdata, ydata):
    x1 = int(xdata - 15)
    x2 = int(xdata + 125)
    y1 = int(ydata - 15)
    y2 = int(ydata + 15)
    crop_box = (x1, y1, x2, y2)
    return crop_box


def draw_box_and_show_image(issue, image, crop_box):
    # draw box on 
    drawing = issue.draw_box(image.copy(), crop_box)
    drawing_rgb = cv2.cvtColor(drawing, cv2.COLOR_BGR2RGB)
    image_ax.imshow(drawing_rgb)
    fig.canvas.draw()
    return drawing
    

def process_clicked_in_image(crop_box):
    
    # expand x/y points back out for full resolution image by dividing
    # by reduction ratio
    print(f'crop_box before: [crop_box]')
    crop_box = [x/resize_ratio for x in crop_box]
    print(f'crop_box after: [crop_box]')
    
    

def onclick_select(event):
    
    if event.inaxes == next_ax:
        # clicked on Next button
        print('NEXT!')
    elif event.inaxes == previous_ax:
        # clicked on Previous button
        print('!PREVIOUS')
    elif event.inaxes == image_ax:
        
        # clicked in picture area
        print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
              ('double' if event.dblclick else 'single', event.button,
               event.x, event.y, event.xdata, event.ydata))
        
        # create crop box based on click data
        date_crop_box = create_crop_box_from_click_data(event.xdata, event.ydata)
        
        # draw box on image and show it
        drawing = draw_box_and_show_image(issue, image, crop_box)
            
        # draw box on image, then convert to RGB and show
        drawing = issue.draw_box(image_resized, date_crop_box)
        drawing = cv2.cvtColor(drawing, cv2.COLOR_BGR2RGB)
        image_ax.imshow(drawing)
        fig.canvas.draw()

        # expand x/y points back out for full resolution image by dividing by
        # the reduction ratio
        x1_original = int(x1/resize_ratio)
        y1_original = int(y1/resize_ratio)
        x2_original = int(x2/resize_ratio)
        y2_original = int(y2/resize_ratio)
        
        # create crop_box with points based on the original image
        crop_box_original = (x1_original, y1_original, x2_original, y2_original)
        
        # ocr the clicked area
        image_cropped_for_ocr = issue.box_crop(crop_box_original, issue.image)
        binarized = issue.binarize(image=image_cropped_for_ocr)
        line_and_word_boxes, text = issue.ocr(image=binarized)

        # it's possible there's more than 1 line of text
        # so make each one a possible date
        for line in line_and_word_boxes:
            # print(line.content)

            # replace new lines with spaces
            line = line.content.replace('\n', ' ')
            # print(line)
            # strip periods, single/double-quotes, and spaces at the beginning and end of the line
            line = line.strip('.').strip(' ').strip("'").strip('"')
            # print(line)

            # many of the lines are obviously not dates
            # so only add lines/crop_boxes if
            # the line contains any numbers
            for char in line:
                if char.isdigit():
                    crop_and_text_dict[line] = date_crop_box
                    break

        # process OCR results

        count_for_csv = 0
        date_guess = None

        for date in crop_and_text_dict:

            parsed_date = None
            month = None


            if len(date) < 5:
                continue
            try:
                # print(f'Processing: "{date}"')
                parsed_date = parse(date)

            except ValueError:  # couldn't parse the date
                # split on spaces and look for each word in the months dictionary
                words = date.split(' ')
                for i, word in enumerate(words):

                    word = str(word)  # cast word as a string in case of weirdness

                    # only process word results over 2 chars in length
                    if len(word) > 2:

                        best_word = word

                        # try to match the word as lower-case with the months dictionary
                        # returns None on failed search
                        month = search(months_dict, word.lower())

                        if month is None:  # try autocorrecting the word and searching again
                            fixed_word = spell(word)
                            # print(f'fixed_word: {fixed_word}')
                            month = search(months_dict, fixed_word.lower())
                            best_word = fixed_word

                        # try adding the previous word to every word after the first and re-searching
                        if i > 0 and month is None:
                            word_plus_word_before = words[i-1] + word
                            month = search(
                                months_dict, word_plus_word_before.lower())
                            best_word = word_plus_word_before

                        # try adding the next word to every word but the last
                        if i+1 < (len(words)) and month is None:
                            word_plus_word_after = word + words[i+1]
                            month = search(
                                months_dict, word_plus_word_after.lower())
                            best_word = word_plus_word_after

                        if month is None:  # continue to the next word
                            continue
                        else:
                            # print(f'word guess for {month} from best word: {best_word}')
                            # set day_year to index + 1 to the end of words list
                            # join the words list and add the space back
                            year = ' '.join(words[i+1:])

                            corrected_date = f'{month} {year}'
                            # print(f'after: {corrected_date}')
                            try:
                                parsed_date = parse(corrected_date)
                                # print(f'parsed date: {parsed_date}')
                            except ValueError:
                                try:
                                    parsed_date = parse(corrected_date, fuzzy=True)
                                    # print(f'fuzzy parsed date: {parsed_date}')
                                except ValueError:
                                    # parsed_date = "Date couldn't be parsed"
                                    continue

            if parsed_date is None:
                # if we found a month go ahead and post the text value
                # of the found date as it may be correct
                # print(f'month in parsed_data is None: {month}')
                if month:
                    date_guess = str(date)
                else:
                    date_guess = None

            # if date after 2000 just pass the text
            elif int(parsed_date.strftime("%Y")) > 1999:
                date_guess = str(date)
            #     if month:
            #         date_guess = str(date)
            #     else:
            #         date_guess = None

            else:
                # print(f'Type of parsed_date: {type(parsed_date)}')
                date_guess = parsed_date.strftime("%B %Y")
                month = parsed_date.strftime("%B")
                year = parsed_date.strftime("%Y")
                # if first digit of year is 0, change it to 1
                if int(year[0]) == 0:
                    # print(f'year <1000: {year}')
                    year = '1' + year[1:]
                    # print(f'year fixed: {year}')
                    # then rebuild date_guess
                    date_guess = f'{month} {year}'
                elif int(year[1]) != 9:
                    year = '19' + year[2:]
                    # then rebuild date_guess
                    date_guess = f'{month} {year}'

    #         crop_box = crop_and_text_dict[date]

    #         x1, y1, x2, y2 = crop_box

        # draw a black rectangle above this to add text to
        drawing = cv2.rectangle(drawing, (x1, y1-25), (x2, y1), (0, 0, 0), -1)  # negative thickness to fill rectangle

        font = cv2.FONT_HERSHEY_SIMPLEX
        if date_guess is not None:
            date_writing = date_guess
        else:
            date_writing = text #'Date Guess Fail'
        drawing = cv2.putText(drawing, date_writing, (x1+5, (y1-5)), font, 0.5, (255,255,255), 1, cv2.LINE_AA)
        image_ax.imshow(drawing)

        # ax[1].clear()
        # if date_guess is not None:
        #     ax[1].text(0.1, 0.5, s=(f'Date guess: {date_guess}'), bbox=dict(facecolor='red', alpha=0.5))
        # else:
        #     ax[1].text(0.1, 0.5, s=(f'OCR text (date FAIL): {line}'), bbox=dict(facecolor='red', alpha=0.5))
        fig.canvas.draw()

cid = fig.canvas.mpl_connect('button_press_event', onclick_select)
plt.tight_layout()
plt.show()

In [ ]:
# create pop-up image and choose where to OCR on it
from matplotlib.widgets import Button

# create matplotlib figure and axis
fig, ax = plt.subplots()

# subplot the figure with gridspec
gs = fig.add_gridspec(10,10)
image_ax = fig.add_subplot(gs[0:8, :])
previous_ax = fig.add_subplot(gs[9, 8])
next_ax = fig.add_subplot(gs[9, 9])

# issue.preprocess_image()
#     image_resized = Image.fromarray(issue.image).convert('L')
#     width, height = image_resized.size
#     image_resized = image_resized.resize((int(width*0.25), int(height*0.25)))
#     image_resized = np.array(image_resized)
#     image_resized = np.stack((image_resized,)*3, axis=-1)
#     image_ax.imshow(image_resized, cmap='gray')
#     fig.canvas.draw()
    
#     return image_resized

def resize_issue_for_qt_popup(issue):
    image_resized, resize_ratio = resize(issue.image, width=issue.config['resize_width'])
    return image_resized, resize_ratio
    
    
# initial data
page_1_paths_dictionary = {path.name: path for path in page_1_paths_list}
initial_image_name = '0012_004268_000038_0001.tif'


# Create widgets
image_name_widget = widgets.Dropdown(
    options=list(page_1_paths_dictionary), value=initial_image_name)
bprev = Button(previous_ax, 'Previous', hovercolor='green')
bnext = Button(next_ax, 'Next', hovercolor='green')

def choose_image(image_name):
    # Let's store the result in the global `image` that we can then use in our image editor below
    global image_resized, resize_ratio
    try:
        image_path = page_1_paths_dictionary[image_name.new]
    except AttributeError:  # 'str' object has no attribute 'new' if it's the first image loaded
        image_path = page_1_paths_dictionary[image_name]
    
    issue = TnFarmNews(image_path)
    # print(issue.image_path.name)
    
    image_resized, resize_ratio = resize_issue_for_qt_popup(issue)
    
    image_ax.imshow(image_resized, cmap='gray')
    
    fig.canvas.draw()
    
    return image_resized, resize_ratio


image_name_widget.observe(choose_image, 'value')


interact(choose_image, image_name=[x.name for x in page_1_paths_list])



def create_crop_box_from_click_data(xdata, ydata):
    x1 = int(xdata - 15)
    x2 = int(xdata + 125)
    y1 = int(ydata - 15)
    y2 = int(ydata + 15)
    crop_box = (x1, y1, x2, y2)
    return crop_box


def draw_box_and_show_image(issue, image, crop_box):
    # draw box on 
    drawing = issue.draw_box(image.copy(), crop_box)
    drawing_rgb = cv2.cvtColor(drawing, cv2.COLOR_BGR2RGB)
    image_ax.imshow(drawing_rgb)
    fig.canvas.draw()
    return drawing
    

def ocr_clicked_image_area(crop_box):
    
    # expand x/y points back out for full resolution image by dividing
    # by reduction ratio
    print(f'crop_box before: [crop_box]')
    crop_box = [x/resize_ratio for x in crop_box]
    print(f'crop_box after: [crop_box]')
    
    image_cropped_for_ocr = issue.box_crop(crop_box, issue.image)
    binarized = issue.binarize(image_cropped_for_ocr)
    line_and_word_boxes, _ = issue.ocr(binarized)
    
    return line_and_word_boxes 
    

def onclick_select(event):
    
    if event.inaxes == next_ax:
        # clicked on Next button
        print('NEXT!')
    elif event.inaxes == previous_ax:
        # clicked on Previous button
        print('!PREVIOUS')
    elif event.inaxes == image_ax:
        
        # print data from clicked in picture area
        print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
              ('double' if event.dblclick else 'single', event.button,
               event.x, event.y, event.xdata, event.ydata))
        
        # create crop box based on click data
        date_crop_box = create_crop_box_from_click_data(event.xdata, event.ydata)
        
        # draw box on image and display it
        drawing = draw_box_and_show_image(issue, image, crop_box)
        
        # create crop_box for the original using resize_ratio
        crop_box_original = progress_clicked_in_image(date_crop_box)
        
        # ocr the clicked area
        line_and_word_boxes = ocr_clicked_image_area(crop_box_original)

        # it's possible there's more than 1 line of text
        # so make each one a possible date
        for line in line_and_word_boxes:
            # print(line.content)

            # replace new lines with spaces
            line = line.content.replace('\n', ' ')
            # print(line)
            # strip periods, single/double-quotes, and spaces at the beginning and end of the line
            line = line.strip('.').strip(' ').strip("'").strip('"')
            # print(line)

            # many of the lines are obviously not dates
            # so only add lines/crop_boxes if
            # the line contains any numbers
            for char in line:
                if char.isdigit():
                    crop_and_text_dict[line] = date_crop_box
                    break

        # process OCR results

        count_for_csv = 0
        date_guess = None

        for date in crop_and_text_dict:

            parsed_date = None
            month = None


            if len(date) < 5:
                continue
            try:
                # print(f'Processing: "{date}"')
                parsed_date = parse(date)

            except ValueError:  # couldn't parse the date
                # split on spaces and look for each word in the months dictionary
                words = date.split(' ')
                for i, word in enumerate(words):

                    word = str(word)  # cast word as a string in case of weirdness

                    # only process word results over 2 chars in length
                    if len(word) > 2:

                        best_word = word

                        # try to match the word as lower-case with the months dictionary
                        # returns None on failed search
                        month = search(months_dict, word.lower())

                        if month is None:  # try autocorrecting the word and searching again
                            fixed_word = spell(word)
                            # print(f'fixed_word: {fixed_word}')
                            month = search(months_dict, fixed_word.lower())
                            best_word = fixed_word

                        # try adding the previous word to every word after the first and re-searching
                        if i > 0 and month is None:
                            word_plus_word_before = words[i-1] + word
                            month = search(
                                months_dict, word_plus_word_before.lower())
                            best_word = word_plus_word_before

                        # try adding the next word to every word but the last
                        if i+1 < (len(words)) and month is None:
                            word_plus_word_after = word + words[i+1]
                            month = search(
                                months_dict, word_plus_word_after.lower())
                            best_word = word_plus_word_after

                        if month is None:  # continue to the next word
                            continue
                        else:
                            # print(f'word guess for {month} from best word: {best_word}')
                            # set day_year to index + 1 to the end of words list
                            # join the words list and add the space back
                            year = ' '.join(words[i+1:])

                            corrected_date = f'{month} {year}'
                            # print(f'after: {corrected_date}')
                            try:
                                parsed_date = parse(corrected_date)
                                # print(f'parsed date: {parsed_date}')
                            except ValueError:
                                try:
                                    parsed_date = parse(corrected_date, fuzzy=True)
                                    # print(f'fuzzy parsed date: {parsed_date}')
                                except ValueError:
                                    # parsed_date = "Date couldn't be parsed"
                                    continue

            if parsed_date is None:
                # if we found a month go ahead and post the text value
                # of the found date as it may be correct
                # print(f'month in parsed_data is None: {month}')
                if month:
                    date_guess = str(date)
                else:
                    date_guess = None

            # if date after 2000 just pass the text
            elif int(parsed_date.strftime("%Y")) > 1999:
                date_guess = str(date)
            #     if month:
            #         date_guess = str(date)
            #     else:
            #         date_guess = None

            else:
                # print(f'Type of parsed_date: {type(parsed_date)}')
                date_guess = parsed_date.strftime("%B %Y")
                month = parsed_date.strftime("%B")
                year = parsed_date.strftime("%Y")
                # if first digit of year is 0, change it to 1
                if int(year[0]) == 0:
                    # print(f'year <1000: {year}')
                    year = '1' + year[1:]
                    # print(f'year fixed: {year}')
                    # then rebuild date_guess
                    date_guess = f'{month} {year}'
                elif int(year[1]) != 9:
                    year = '19' + year[2:]
                    # then rebuild date_guess
                    date_guess = f'{month} {year}'

    #         crop_box = crop_and_text_dict[date]

    #         x1, y1, x2, y2 = crop_box

        # draw a black rectangle above this to add text to
        drawing = cv2.rectangle(drawing, (x1, y1-25), (x2, y1), (0, 0, 0), -1)  # negative thickness to fill rectangle

        font = cv2.FONT_HERSHEY_SIMPLEX
        if date_guess is not None:
            date_writing = date_guess
        else:
            date_writing = text #'Date Guess Fail'
        drawing = cv2.putText(drawing, date_writing, (x1+5, (y1-5)), font, 0.5, (255,255,255), 1, cv2.LINE_AA)
        image_ax.imshow(drawing)

        # ax[1].clear()
        # if date_guess is not None:
        #     ax[1].text(0.1, 0.5, s=(f'Date guess: {date_guess}'), bbox=dict(facecolor='red', alpha=0.5))
        # else:
        #     ax[1].text(0.1, 0.5, s=(f'OCR text (date FAIL): {line}'), bbox=dict(facecolor='red', alpha=0.5))
        fig.canvas.draw()

cid = fig.canvas.mpl_connect('button_press_event', onclick_select)
plt.tight_layout()
plt.show()

In [ ]:
# create pop-up image and choose where to OCR on it
from matplotlib.widgets import Button


issue = TnFarmNews(image_path)
issue.preprocess_image()
image_resized = Image.fromarray(issue.image_cropped).convert('L')
width, height = image_resized.size
image_resized = image_resized.resize((int(width*0.25), int(height*0.25)))
image_resized_cv2 = np.array(image_resized)
image_resized_cv2 = np.stack((image_resized_cv2,)*3, axis=-1)

# create matplotlib window
fig, ax = plt.subplots()

gs = fig.add_gridspec(10,10)
image_ax = fig.add_subplot(gs[0:8, :])
previous_ax = fig.add_subplot(gs[9, 8])
next_ax = fig.add_subplot(gs[9, 9])


bprev = Button(previous_ax, 'Previous', hovercolor='green')
bnext = Button(next_ax, 'Next', hovercolor='green')
image_ax.imshow(image_resized, cmap='gray')


def onclick_select(event):
    if event.inaxes == next_ax:
        print('NEXT!')
    elif event.inaxes == previous_ax:
        print('!PREVIOUS')
    elif event.inaxes == image_ax:
        crop_and_text_dict = OrderedDict()
        # print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
        #       ('double' if event.dblclick else 'single', event.button,
        #        event.x, event.y, event.xdata, event.ydata))

        x1 = int(event.xdata - 15)
        x2 = int(event.xdata + 125)
        y1 = int(event.ydata - 15)
        y2 = int(event.ydata + 15)

        # package sub_image's crop_box for dictionary
        date_crop_box = (x1, y1, x2, y2)
        drawing = cv2.rectangle(image_resized_cv2.copy(),
                                (x1, y1), (x2, y2), (0, 0, 255), 2)
        drawing = cv2.cvtColor(drawing, cv2.COLOR_BGR2RGB)

        image_ax.imshow(drawing)
        fig.canvas.draw()

        # expand x/y points back out for full resolution image
        x1_original = int(x1/0.25)
        y1_original = int(y1/0.25)
        x2_original = int(x2/0.25)
        y2_original = int(y2/0.25)
        # ocr the clicked area

        crop_for_ocr = issue.image_cropped[y1_original:y2_original, x1_original:x2_original]
        binarized = issue.binarize(crop_for_ocr)
        line_and_word_boxes, text = issue.ocr(binarized)

        # it's possible there's more than 1 line of text
        # so make each one a possible date
        for line in line_and_word_boxes:
            # print(line.content)

            # replace new lines with spaces
            line = line.content.replace('\n', ' ')
            # print(line)
            # strip periods, single/double-quotes, and spaces at the beginning and end of the line
            line = line.strip('.').strip(' ').strip("'").strip('"')
            # print(line)

            # many of the lines are obviously not dates
            # so only add lines/crop_boxes if
            # the line contains any numbers
            for char in line:
                if char.isdigit():
                    crop_and_text_dict[line] = date_crop_box
                    break

        # process OCR results

        count_for_csv = 0
        date_guess = None

        for date in crop_and_text_dict:

            parsed_date = None
            month = None


            if len(date) < 5:
                continue
            try:
                # print(f'Processing: "{date}"')
                parsed_date = parse(date)

            except ValueError:  # couldn't parse the date
                # split on spaces and look for each word in the months dictionary
                words = date.split(' ')
                for i, word in enumerate(words):

                    word = str(word)  # cast word as a string in case of weirdness

                    # only process word results over 2 chars in length
                    if len(word) > 2:

                        best_word = word

                        # try to match the word as lower-case with the months dictionary
                        # returns None on failed search
                        month = search(months_dict, word.lower())

                        if month is None:  # try autocorrecting the word and searching again
                            fixed_word = spell(word)
                            # print(f'fixed_word: {fixed_word}')
                            month = search(months_dict, fixed_word.lower())
                            best_word = fixed_word

                        # try adding the previous word to every word after the first and re-searching
                        if i > 0 and month is None:
                            word_plus_word_before = words[i-1] + word
                            month = search(
                                months_dict, word_plus_word_before.lower())
                            best_word = word_plus_word_before

                        # try adding the next word to every word but the last
                        if i+1 < (len(words)) and month is None:
                            word_plus_word_after = word + words[i+1]
                            month = search(
                                months_dict, word_plus_word_after.lower())
                            best_word = word_plus_word_after

                        if month is None:  # continue to the next word
                            continue
                        else:
                            # print(f'word guess for {month} from best word: {best_word}')
                            # set day_year to index + 1 to the end of words list
                            # join the words list and add the space back
                            year = ' '.join(words[i+1:])

                            corrected_date = f'{month} {year}'
                            # print(f'after: {corrected_date}')
                            try:
                                parsed_date = parse(corrected_date)
                                # print(f'parsed date: {parsed_date}')
                            except ValueError:
                                try:
                                    parsed_date = parse(corrected_date, fuzzy=True)
                                    # print(f'fuzzy parsed date: {parsed_date}')
                                except ValueError:
                                    # parsed_date = "Date couldn't be parsed"
                                    continue

            if parsed_date is None:
                # if we found a month go ahead and post the text value
                # of the found date as it may be correct
                # print(f'month in parsed_data is None: {month}')
                if month:
                    date_guess = str(date)
                else:
                    date_guess = None

            # if date after 2000 just pass the text
            elif int(parsed_date.strftime("%Y")) > 1999:
                date_guess = str(date)
            #     if month:
            #         date_guess = str(date)
            #     else:
            #         date_guess = None

            else:
                # print(f'Type of parsed_date: {type(parsed_date)}')
                date_guess = parsed_date.strftime("%B %Y")
                month = parsed_date.strftime("%B")
                year = parsed_date.strftime("%Y")
                # if first digit of year is 0, change it to 1
                if int(year[0]) == 0:
                    # print(f'year <1000: {year}')
                    year = '1' + year[1:]
                    # print(f'year fixed: {year}')
                    # then rebuild date_guess
                    date_guess = f'{month} {year}'
                elif int(year[1]) != 9:
                    year = '19' + year[2:]
                    # then rebuild date_guess
                    date_guess = f'{month} {year}'

    #         crop_box = crop_and_text_dict[date]

    #         x1, y1, x2, y2 = crop_box

        # draw a black rectangle above this to add text to
        drawing = cv2.rectangle(drawing, (x1, y1-25), (x2, y1), (0, 0, 0), -1)  # negative thickness to fill rectangle

        font = cv2.FONT_HERSHEY_SIMPLEX
        print(f'date_guess before fail: {date_guess}')
        if date_guess is not None:
            date_writing = date_guess
        else:
            date_writing = 'Date Guess Fail'
        drawing = cv2.putText(drawing, date_writing, (x1+5, (y1-5)), font, 0.5, (255,255,255), 1, cv2.LINE_AA)
        image_ax.imshow(drawing)


        # ax[1].clear()
        # if date_guess is not None:
        #     ax[1].text(0.1, 0.5, s=(f'Date guess: {date_guess}'), bbox=dict(facecolor='red', alpha=0.5))
        # else:
        #     ax[1].text(0.1, 0.5, s=(f'OCR text (date FAIL): {line}'), bbox=dict(facecolor='red', alpha=0.5))
        fig.canvas.draw()


cid = fig.canvas.mpl_connect('button_press_event', onclick_select)
plt.tight_layout()
plt.show()

In [ ]:
# create pop-up image and choose where to OCR on it take 1

image_resized = Image.fromarray(issue.image_cropped).convert('L')
width, height = image_resized.size
image_resized = image_resized.resize((int(width*0.25), int(height*0.25)))
image_resized_cv2 = np.array(image_resized)
image_resized_cv2 = np.stack((image_resized_cv2,)*3, axis=-1)
fig, axes = plt.subplots(nrows=2, ncols=1)
axes[0].imshow(image_resized_cv2, cmap='gray')
axes[1].text(0.45, 0.5, 'Click on the first letter of the month',
             bbox=dict(facecolor='red', alpha=0.5))
plt.tight_layout()


def onclick(event):
    crop_and_text_dict = OrderedDict()
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))
    
    x1 = int(event.xdata - 15)
    x2 = int(event.xdata + 115)
    y1 = int(event.ydata - 15)
    y2 = int(event.ydata + 15)

    # package sub_image's crop_box for dictionary
    date_crop_box = (x1, y1, x2, y2)
    drawing = cv2.rectangle(image_resized_cv2.copy(),
                            (x1, y1), (x2, y2), (0, 0, 255), 2)
    drawing = cv2.cvtColor(drawing, cv2.COLOR_BGR2RGB)
    axes[0].imshow(drawing)
    axes[1].clear()
    axes[1].text(0.1, 0.5, s=(f'Starting OCR at "({x1}, {y1}), ({x2}, {y2})"'), bbox=dict(facecolor='red', alpha=0.5))
    fig.canvas.draw()
    
    # expand x/y points back out for full resolution image
    x1 = int(x1/0.25)
    y1 = int(y1/0.25)
    x2 = int(x2/0.25)
    y2 = int(y2/0.25)
    # ocr the clicked area
    
    crop_for_ocr = issue.image_cropped[y1:y2, x1:x2]
    binarized = issue.binarize(crop_for_ocr)
    line_and_word_boxes, text = issue.ocr(binarized)

    # it's possible there's more than 1 line of text
    # so make each one a possible date
    for line in line_and_word_boxes:
        # print(line.content)

        # replace new lines with spaces
        line = line.content.replace('\n', ' ')
        # print(line)
        # strip periods, single/double-quotes, and spaces at the beginning and end of the line
        line = line.strip('.').strip(' ').strip("'").strip('"')
        # print(line)

        # many of the lines are obviously not dates
        # so only add lines/crop_boxes if
        # the line contains any numbers
        for char in line:
            if char.isdigit():
                crop_and_text_dict[line] = date_crop_box
                break

    # process OCR results

    count_for_csv = 0
    date_guess = None

    for date in crop_and_text_dict:

        parsed_date = None
        month = None
        

        if len(date) < 5:
            continue
        try:
            # print(f'Processing: "{date}"')
            parsed_date = parse(date)

        except ValueError:  # couldn't parse the date
            # split on spaces and look for each word in the months dictionary
            words = date.split(' ')
            for i, word in enumerate(words):

                word = str(word)  # cast word as a string in case of weirdness

                # only process word results over 2 chars in length
                if len(word) > 2:

                    best_word = word

                    # try to match the word as lower-case with the months dictionary
                    # returns None on failed search
                    month = search(months_dict, word.lower())

                    if month is None:  # try autocorrecting the word and searching again
                        fixed_word = spell(word)
                        # print(f'fixed_word: {fixed_word}')
                        month = search(months_dict, fixed_word.lower())
                        best_word = fixed_word

                    # try adding the previous word to every word after the first and re-searching
                    if i > 0 and month is None:
                        word_plus_word_before = words[i-1] + word
                        month = search(
                            months_dict, word_plus_word_before.lower())
                        best_word = word_plus_word_before

                    # try adding the next word to every word but the last
                    if i+1 < (len(words)) and month is None:
                        word_plus_word_after = word + words[i+1]
                        month = search(
                            months_dict, word_plus_word_after.lower())
                        best_word = word_plus_word_after

                    if month is None:  # continue to the next word
                        continue
                    else:
                        # print(f'word guess for {month} from best word: {best_word}')
                        # set day_year to index + 1 to the end of words list
                        # join the words list and add the space back
                        year = ' '.join(words[i+1:])

                        corrected_date = f'{month} {year}'
                        # print(f'after: {corrected_date}')
                        try:
                            parsed_date = parse(corrected_date)
                            # print(f'parsed date: {parsed_date}')
                        except ValueError:
                            try:
                                parsed_date = parse(corrected_date, fuzzy=True)
                                # print(f'fuzzy parsed date: {parsed_date}')
                            except ValueError:
                                # parsed_date = "Date couldn't be parsed"
                                continue

        if parsed_date is None:
            # if we found a month go ahead and post the text value
            # of the found date as it may be correct
            # print(f'month in parsed_data is None: {month}')
            if month:
                date_guess = str(date)
            else:
                date_guess = None

        # if date after 2000 just pass the text
        elif int(parsed_date.strftime("%Y")) > 1999:
            date_guess = str(date)
        #     if month:
        #         date_guess = str(date)
        #     else:
        #         date_guess = None

        else:
            # print(f'Type of parsed_date: {type(parsed_date)}')
            date_guess = parsed_date.strftime("%B %Y")
            month = parsed_date.strftime("%B")
            year = parsed_date.strftime("%Y")
            # if first digit of year is 0, change it to 1
            if int(year[0]) == 0:
                # print(f'year <1000: {year}')
                year = '1' + year[1:]
                # print(f'year fixed: {year}')
                # then rebuild date_guess
                date_guess = f'{month} {year}'
            elif int(year[1]) != 9:
                year = '19' + year[2:]
                # then rebuild date_guess
                date_guess = f'{month} {year}'

        crop_box = crop_and_text_dict[date]

        x1, y1, x2, y2 = crop_box

    axes[1].clear()
    if date_guess is not None:
        axes[1].text(0.1, 0.5, s=(f'Date guess: {date_guess}'), bbox=dict(facecolor='red', alpha=0.5))
    else:
        axes[1].text(0.1, 0.5, s=(f'OCR text (date FAIL): {line}'), bbox=dict(facecolor='red', alpha=0.5))
    fig.canvas.draw()


cid = fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()

In [ ]:
len(processed_df)

In [ ]:
processed_df

In [ ]:
test_date = date.strip('"')#.strip('.')# date.replace(',', '').replace('.', '').replace('"', '')
test_date

In [ ]:
parse(test_date)

In [ ]:
print(date.replace('\n', ' '))

In [ ]:
date_csv_path = data_dir_path.joinpath('TnFarmNews_date.csv')
if date_csv_path.is_file():
    processed_df = pd.read_csv(date_csv_path)
else:
    processed_df = pd.DataFrame()
processed_df

In [ ]:
# anything in the new fail list not in the old fail list?
test = fail_df.merge(old_fail_df, how='outer', indicator=True).query(
    '_merge == "left_only"').drop('_merge', 1)
test.reset_index(drop=True)

In [ ]:
image_path = crop_df['image_path'][0]
image = cv2.imread(str(image_path))
x1, y1, x2, y2 = crop_df['title_crop'][0]
image_cropped = image[y1:y2, x1:x2]
plt.imshow(image), plt.show()
plt.imshow(image_cropped), plt.show()

In [ ]:
@interact
def show_images(file=page_1_paths_list):
    # load image
    issue = TnFarmNews(file)
    issue.ocr()
    # draw_month_box(issue.line_and_word_boxes, issue.image_cropped)
    # image = Image.open(file)
    # temp_image_path = Path('_temp_image.jpg')
    # image.save(temp_image_path)
    # display(ipyImage(temp_image_path))
    issue.guess_title(title_crops_dict, debug=True)
    print(issue.best_title)

In [ ]:
# this is my attempt to try and make a crop-value widget!

page_1_paths_dictionary = {path.name: path for path in page_1_paths_list}
initial_path = page_1_paths_list[0]
initial_images_key = initial_path.name
initial_image = cv2.imread(image)
initial_x2 =

# Create widgets
images = widgets.Dropdown(
    options=page_1_paths_dictionary, value=initial_path_key)
crop_x1 = widgets.IntRangeSlider(value=(100, , min=0, max=initial_image.shape[1])
crop_y1=widgets.IntSlider(value=0, min=0, max=initial_image.shape[0])
crop_x2=widgets.IntSlider()

# Updates the image options based on directory value
def update_crop(*args):
    crop_x1.max=os.listdir(directory.value)

# Tie the image options to directory value
directory.observe(update_crop, 'value')

# Show the images
def show_images(fdir, file):
    display(Image(f'{fdir}/{file}'))

_=interact(show_images, fdir=directory, file=images)
def crop_image(file=page_1_paths_list, x1=(0, Image.open(file).size[0])):
    # load image
    issue=TnFarmNews(file)
    issue.ocr()
    draw_month_box(issue.line_and_word_boxes, issue.image_cropped)
    # image = Image.open(file)
    # temp_image_path = Path('_temp_image.jpg')
    # image.save(temp_image_path)
    # display(ipyImage(temp_image_path))
    issue.guess_title(debug=True)
    print(issue.best_title)